In [16]:
import sys
sys.path.append('../..')
from src.models.train import *

%run -i 'train.py'

Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 4s 227us/sample - loss: 34.1766 - accuracy: 0.2596 - val_loss: 33.7593 - val_accuracy: 0.0000e+00
Epoch 2/50
16000/16000 [==============================] - 3s 166us/sample - loss: 33.5229 - accuracy: 0.2521 - val_loss: 33.1369 - val_accuracy: 0.0020
Epoch 3/50
16000/16000 [==============================] - 3s 170us/sample - loss: 32.1087 - accuracy: 0.2548 - val_loss: 32.2976 - val_accuracy: 0.0000e+00
Epoch 4/50
16000/16000 [==============================] - 3s 167us/sample - loss: 30.8706 - accuracy: 0.2954 - val_loss: 32.2962 - val_accuracy: 0.6370
Epoch 5/50
16000/16000 [==============================] - 2s 139us/sample - loss: 30.3465 - accuracy: 0.3492 - val_loss: 33.0091 - val_accuracy: 0.5067
Epoch 6/50
16000/16000 [==============================] - 2s 156us/sample - loss: 30.2394 - accuracy: 0.3761 - val_loss: 32.5991 - val_accuracy: 0.5490
Epoch 7/50
16000/16000 [=======

Epoch 4/50
16000/16000 [==============================] - 2s 138us/sample - loss: 30.8731 - accuracy: 3.1250e-04 - val_loss: 32.4494 - val_accuracy: 0.0000e+00
Epoch 5/50
16000/16000 [==============================] - 2s 136us/sample - loss: 30.4635 - accuracy: 0.0385 - val_loss: 32.1076 - val_accuracy: 0.0000e+00
Epoch 6/50
16000/16000 [==============================] - 2s 136us/sample - loss: 30.0728 - accuracy: 0.1634 - val_loss: 32.2075 - val_accuracy: 0.0000e+00
Epoch 7/50
16000/16000 [==============================] - 2s 136us/sample - loss: 29.9253 - accuracy: 0.2018 - val_loss: 32.1220 - val_accuracy: 0.0000e+00
Epoch 8/50
16000/16000 [==============================] - 2s 138us/sample - loss: 29.8588 - accuracy: 0.2276 - val_loss: 32.3482 - val_accuracy: 5.0000e-04
Epoch 9/50
16000/16000 [==============================] - 2s 138us/sample - loss: 29.7649 - accuracy: 0.2540 - val_loss: 32.4277 - val_accuracy: 0.0000e+00
Epoch 10/50
16000/16000 [==============================] - 2